In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c
import ipywidgets as widgets
from IPython.display import display

# Parameters 
# 780 nm Toptica DFB Pro L diode lasers, mode hop-free tuning range up to 1400 GHz 
# https://www.toptica.com/products/tunable-diode-lasers/ecdl-dfb-lasers/dfb-pro

# 780 nm
f1 = c / 780e-9  # [Hz] ≈ 3.846e14 Hz

# 1.8 THz difference frequency tuning (TeraScan780)
THz_max = 1.8 # [THz]

def lorentzian(f, f0, sigma_FWHM):
    # Lorentzian lineshape with full-width at half-maximum (FWHM) = sigma_FWHM
    # y = y0 + A / (wsqrt(pi/2)) * e(-2(x-x0)^2/w^2)
    gamma = sigma_FWHM / 2.0
    return (gamma / np.pi) / ((f - f0)**2 + gamma**2)

# using 5 MHz typical instantaneous linewidth from DFB Pro specs
sigma_IR = 5e6 # [Hz]

# assuming Lorentzian model of diode laser linewidths, THz linewidths is the sum of diode laser linewidths
# Assumption based on Liu 2025, Figure 6 https://doi.org/10.3390/mi16090990
sigma_thz = 2 * sigma_IR  # [Hz]

# common scaling factor to exaggerate linewidths for visualization, preserving their ratio
linewidth_scale = 1e3  # makes 5 MHz show up as 5 GHz

# compute a fixed y-limit for the IR plot 
peak_single = lorentzian(np.array([f1]), f1, sigma_IR * linewidth_scale)[0]  # amplitude of one Lorentzian
ymax_IR = 1.2 * peak_single 

# define frequency axes 
IR_span = 5e12  # +-5 THz
N_IR = 1_000_001
f_IR = f1 + np.linspace(-IR_span, IR_span, N_IR)
f_IR_THz = f_IR / 1e12
N_thz = 4001
f_thz = np.linspace(0, THz_max * 1e12, N_thz)
f_thz_THz = f_thz / 1e12

# update function called when slider is moved
def update(delta_THz):
    dT = delta_THz * 1e12  # convert THz to Hz
    
    # build and plot figure
    # stack subplots vertically 
    fig, (ax_IR, ax_thz) = plt.subplots(2, 1, figsize=(7, 7), sharex=False)

    # initial IR spectra for the chosen detuning
    f2 = f1 + dT
    spec_IR = (
        lorentzian(f_IR, f1, sigma_IR * linewidth_scale)
        + lorentzian(f_IR, f2, sigma_IR * linewidth_scale)
    )

    # plot IR spectrum
    (line_IR,) = ax_IR.plot(
        f_IR_THz,
        spec_IR,
        color="tab:blue",
        label=r"$\Delta f = %.2f$ THz" % delta_THz,
    )

    ax_IR.set_xlabel("IR frequency (THz)")
    ax_IR.set_ylabel("Intensity (arb. units)")
    ax_IR.set_title("IR domain (two 780 nm DFB pro L lasers)")
    # IR axis: around 780 nm (~384.6 THz) +- a few THz
    ax_IR.set_xlim(384, 387)
    ax_IR.set_ylim(0, ymax_IR)
    ax_IR.legend(fontsize=8)
    ax_IR.grid(alpha=0.3)    

    # initial THz beat spectrum for the chosen detuning
    spec_thz = lorentzian(f_thz, dT, sigma_thz * linewidth_scale)

    (line_thz,) = ax_thz.plot(
        f_thz_THz,
        spec_thz,
        color="tab:blue",
    )

    ax_thz.set_xlabel("THz frequency (THz)")
    ax_thz.set_ylabel("Intensity (arb. units)")
    ax_thz.set_title("Beat frequency (photomixed output, TeraScan 780)")
    ax_thz.grid(alpha=0.3)
    ax_thz.set_xlim(0, THz_max)

    plt.tight_layout()
    plt.show()

# slider so the user can pick the frequency f2
delta_init_THz = THz_max / 2.0
delta_slider = widgets.FloatSlider(
    value=delta_init_THz,
    min=0.0,
    max=THz_max,   
    step=0.01,
    description=r'$\Delta f$ (THz)',
    continuous_update=True
)

out = widgets.interactive_output(update, {'delta_THz': delta_slider})
display(delta_slider, out)


FloatSlider(value=0.9, description='$\\Delta f$ (THz)', max=1.8, step=0.01)

Output()